**Getting Started with Corvic using Colab**
In this example, we will accomplish a simple query to Corvic. This example is similar to the one mentioned [here](https://app.corvic.ai/docs/mcpIntegrations/openaiMcpWithCorvic) wherein we enquire about video game sales

The following two steps are involved:
<p>
STEP 1: Invoke Corvic agent using the Model Context Protocol and ask a question about video game sales
</p>
<p>
STEP 2: (Optional) Parse the markdown and separate out the text and image data
</p>



---



STEP 1: Invoke Corvic agent using the Model Context Protocol and ask a question about video game sales
<p/>
Please note:
The MCP library is used for corvic agent invocation. Corvic returns markdown text. We will convert markdown and beautiful soup libraries to parse text and images

In [ ]:
# mcp is used for corvic agent invocation
!pip install mcp
!pip install beautifulsoup4
!pip install markdown

In [ ]:
import asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client


In [ ]:
url="YOUR CORVIC MCP ENDPOINT"
headers={
          "Authorization": "YOUR CORVIC API TOKEN"
        }


In [ ]:
async def run():
    async with sse_client(
            url,
            headers=headers,
    ) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # List available tools
            tools = await session.list_tools()
            print(tools)

            # Call a tool
            result = await session.call_tool(
                "query", arguments={
                    "query_content":
                    "Group all the data by name and find the top titles by "
                    "global sales. Output the name and the total global sales "
                    " in a html tabular format."
                    "Provide a bar chart for the same table with the game title"
                    " on X axis and global sales on Y axis"}
            )
            return result



The following line will invoke corvic and collect the markdown response in the corvic_response field

In [ ]:
corvic_response=await run()

You can write the Corvic response directly in an MD file and view it

In [ ]:
with open("vg_query.md", "a") as file:
  for content in corvic_response.content:
      file.write(content.text)

You can directly view the markdown output as below.

In [ ]:
from IPython.display import Markdown, display
display(Markdown(corvic_response.content[0].text))

This completes Step 1

---

STEP 2: Parse the markdown and display the text and images separately

In [ ]:
from types import NoneType
from bs4 import BeautifulSoup
import markdown
import base64


html_content = markdown.markdown(corvic_response.content[0].text)
html_view = BeautifulSoup("<html>"+html_content+"</html>", features="html.parser")

html_node = html_view.find('html')
#Find images and strip from original html
image_data=[]
text_output=''
for c1 in html_node.children:
  # print(str(type(c1)))
  if 'bs4.element.NavigableString' not in str(type(c1)):
    if len(list(c1.children)) > 0:
      for c2 in c1.children:
        # print(str(type(c2)) + " " + str(c2.name))
        if "img" == str(c2.name):
          b64encoded_img_string=c2.attrs['src'].replace('data:image/png;base64,', '')
          image_data.append(base64.b64decode(b64encoded_img_string))
          c2.decompose()
        else:
          # print(str(type(c2)) )
          if 'NoneType' not in str(type(c2)):
            text_output+=repr(c2)
  else:
    text_output+=repr(c1)

Here is the text output in a html format for better readability

In [ ]:
from IPython.core.display import display, HTML

display(HTML(html_node.prettify()))

Here is the first image from the Corvic response

In [ ]:
from IPython import display

display.Image(image_data[0])



---



Summary: You have been able to invoke the Corvic MCP from this Colab workbook.
You also explored the response in different ways allowing you great flexibility in using the output in your AI pipelines